In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
import random

In [10]:
class MyLineReg:
    #1. Инициализация параметров
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None, reg=None, l1_coef=0, l2_coef=0, sgd_sample=None, random_state=42):
        self.weights = None
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.best_score = None
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.sgd_sample = sgd_sample
        self.random_state = random_state
       
    def __str__(self):
        params = vars(self) # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyLineReg class: {params_str}"
    
    def _calculate_metric(self, y_true, y_pred):
        if self.metric == 'mae':
            return np.mean(np.abs(y_true - y_pred))
        elif self.metric == 'mse':
            return np.mean((y_true - y_pred) ** 2)
        elif self.metric == 'rmse':
            return np.sqrt(np.mean((y_true - y_pred) ** 2))
        elif self.metric == 'mape':
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 
        elif self.metric == 'r2':
            ss_res = np.sum((y_true - y_pred) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            return 1 - (ss_res / ss_tot)
        else:
            return None
    
    def regularization_loss(self, mse):
        if self.reg == 'l1':
            # L1 регуляризация: добавляем сумму абсолютных значений весов
            regularization = self.l1_coef * np.sum(np.abs(self.weights[1:]))
        elif self.reg == 'l2':
            # L2 регуляризация: добавляем сумму квадратов весов
            regularization = self.l2_coef * np.sum(self.weights[1:] ** 2)
        elif self.reg == 'elasticnet':
            # ElasticNet регуляризация: комбинация L1 и L2
            l1_regularization = self.l1_coef * np.sum(np.abs(self.weights[1:]))
            l2_regularization = self.l2_coef * np.sum(self.weights[1:] ** 2)
            regularization = l1_regularization + l2_regularization
        else:
            regularization = 0
        return mse + regularization
    
    def regularization_gradient(self, gradients):
        if self.reg == 'l1':
            gradients[1:] += self.l1_coef * np.sign(self.weights[1:])  # L1 градиент
        elif self.reg == 'l2':
            gradients[1:] += 2 * self.l2_coef * self.weights[1:]  # L2 градиент
        elif self.reg == 'elasticnet':
            gradients[1:] += self.l1_coef * np.sign(self.weights[1:])  # L1 часть
            gradients[1:] += 2 * self.l2_coef * self.weights[1:]  # L2 часть
        return gradients
    
    def _update_learning_rate(self, iteration):
        # динамический learning rate
        return 0.5 * (0.85 ** iteration)
    
    def _sample_batch(self, X: pd.DataFrame, y: pd.Series): 
    #Генерация мини-пакета данных для стохастического градиентного спуска

        n_samples = X.shape[0] #количество строк в наборе X
        
        if isinstance(self.sgd_sample, int):
            sample_size = self.sgd_sample
        elif isinstance(self.sgd_sample, float):
            sample_size = int(n_samples * self.sgd_sample)
        else:
            return X, y  # Если sgd_sample=None, используем весь набор данных
        
        sample_rows_idx = random.sample(range(n_samples), sample_size) #Функция random.sample выбирает sample_size уникальных индексов строк из диапазона от 0 до n_samples - 1 (где n_samples — это общее количество строк в наборе данных).
        return X[sample_rows_idx], y[sample_rows_idx]
    
    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        random.seed(self.random_state)
        
        # 2. Добавляем колонку с единицами слева
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        
        # 3. Определяем количество фичей и создаём вектор весов
        n_features = X_b.shape[1]  # Количество фичей + 1 (для bias))
        self.weights = np.ones(n_features)  # Вектор весов, состоящий из единиц
        
        #4. Цикл для обучения
        for i in range(self.n_iter):
            # Создаём подвыборку данных (даже если не задан sgd_sample)
            X_batch, y_batch = self._sample_batch(X_b, y)
            
            # Рассчитываем предсказания на мини-пакете
            y_pred_batch = X_batch.dot(self.weights)
            error_batch = y_pred_batch - y_batch
            
            # Предсказание значений y y_pred
            y_pred = np.dot(X_b, self.weights)
            
            # Вычисляем ошибку (MSE - Mean Squared Error)
            error = y_pred - y
            mse = np.mean(error ** 2)
            mse = self.regularization_loss(mse)
            
            # # Вычисление градиента для мини-пакета или всей выборки если sgd_Sample = None
            n_samples = X_batch.shape[0]
            gradients = (2 / n_samples) * np.dot(X_batch.T, error_batch)
            # gradients = (2 / len(X)) * np.dot(X_b.T, error)
            gradients = self.regularization_gradient(gradients)
            
            # Динамическое обновление learning_rate
            current_learning_rate = self._update_learning_rate(i)
            
            # Обновляем веса
            self.weights -= current_learning_rate * gradients
            
            # Обновляем веса в противоположную сторону от градиента
            # self.weights -= self.learning_rate * gradients
                
        # Рассчитываем метрику
            if self.metric:
                metric_value = self._calculate_metric(y, y_pred)
                self.best_score = metric_value  # Сохраняем последнее значение метрики
            else:
                metric_value = None

            # Логирование
            if i == 0 and verbose:
                print(f"start | loss: {mse} | {self.metric}: {metric_value}")

            elif verbose and i % verbose == 0:
                print(f"{i} | loss: {mse} | {self.metric}: {metric_value}")
                
    #6. Работа с весами           
    def get_coef(self):
        # Возвращаем веса, начиная со второго элемента (веса без bias, w0)
        if self.weights is not None:
            return self.weights[1:]
        else:
            raise ValueError("Model is not fitted yet. Please call the fit method first.")

    #7. Предсказания
    def predict(self, X: pd.DataFrame):
        # Добавляем столбец единиц для bias (сдвига)
        if self.weights is None:
            raise ValueError("Model is not fitted yet. Please call the fit method first.")
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return np.dot(X_b, self.weights)
    
    def get_best_score(self):
        #Возвращает последнее значение выбранной метрики.
        if self.best_score is not None:
            return self.best_score
        else:
            raise ValueError("Metric is not defined or model is not fitted yet.")

In [24]:
#тест стохастического спуска
X, y = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42)
X = pd.DataFrame(X)
y = pd.Series(y)

# Инициализируем модель
model_stoch = MyLineReg(n_iter=100, learning_rate=0.1, metric='mse', reg='l2', l2_coef=0.1, sgd_sample=0.1, random_state=42)

# Обучаем модель
model_stoch.fit(X, y, verbose=50)

# Выводим результаты
print("Коэффициенты модели:", np.mean(model_stoch.get_coef()))

start | loss: 19062.577956731744 | mse: 19062.077956731744
50 | loss: 1977.9618009681656 | mse: 207.90200292155404
Коэффициенты модели: 56.75978433169247


In [18]:
# Тест динамического learning_rate
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X_df = pd.DataFrame(X)
y_series = pd.Series(y)

# Создаем модель
model = MyLineReg(n_iter=50, learning_rate=0.1, reg=None, metric='mse')

# Обучение модели
model.fit(X_df, y_series, verbose=10)

# Получение результата
print("Последнее значение метрики MSE:", model.get_best_score())

start | loss: 11153.868597347617 | mse: 11153.868597347617
10 | loss: 0.15482041682044387 | mse: 0.15482041682044387
20 | loss: 0.04263319083173665 | mse: 0.04263319083173665
30 | loss: 0.03480673153901547 | mse: 0.03480673153901547
40 | loss: 0.03352418391294136 | mse: 0.03352418391294136
Последнее значение метрики MSE: 0.033290942780611864


In [19]:
# Проверка
X = pd.DataFrame({
    'test_1': [1, 2, 3, 4, 5],
    'test_2': [10, 20, 30, 40, 50]
})
y = pd.Series([5, 7, 9, 11, 13])

# Создаём объект класса MyLineReg с n_iter=50 и learning_rate=0.1
model = MyLineReg(n_iter=50, learning_rate=0.1)

# Обучаем модель
model.fit(X, y)

# Получаем коэффициенты (веса, начиная со второго элемента)
coefficients = model.get_coef()

# Выводим среднее значение коэффициентов
mean_coef = np.mean(coefficients)
print("Среднее значение коэффициентов:", mean_coef)

Среднее значение коэффициентов: 1.442862943168177e+59


In [20]:
# Генерация трех различных наборов данных
X1, y1 = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X2, y2 = make_regression(n_samples=150, n_features=3, noise=0.2, random_state=24)
X3, y3 = make_regression(n_samples=200, n_features=1, noise=0.3, random_state=12)

# Конвертация массивов в DataFrame и Series
X1_df = pd.DataFrame(X1)
X2_df = pd.DataFrame(X2)
X3_df = pd.DataFrame(X3)

y1_series = pd.Series(y1)
y2_series = pd.Series(y2)
y3_series = pd.Series(y3)

# Создаем модель линейной регрессии
model1 = MyLineReg(n_iter=50, learning_rate=0.1)
model2 = MyLineReg(n_iter=50, learning_rate=0.1)
model3 = MyLineReg(n_iter=50, learning_rate=0.1)

# Обучаем модели
model1.fit(X1_df, y1_series)
model2.fit(X2_df, y2_series)
model3.fit(X3_df, y3_series)

# Делаем предсказания
predictions1 = model1.predict(X1_df)
predictions2 = model2.predict(X2_df)
predictions3 = model3.predict(X3_df)

# Суммируем все предсказания
total_predictions_sum = np.sum(predictions1) + np.sum(predictions2) + np.sum(predictions3)

# Выводим сумму всех предсказаний
print("Сумма всех предсказаний:", total_predictions_sum)

Сумма всех предсказаний: -1771.0428869154055


In [21]:
#Тестирование метрик
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X_df = pd.DataFrame(X)
y_series = pd.Series(y)

# Тестирование метрики MAE (Mean Absolute Error)
model_mae = MyLineReg(n_iter=50, learning_rate=0.1, metric='mae')
model_mae.fit(X_df, y_series, verbose=False)
mae_score = model_mae.get_best_score()
print(f"Последнее значение метрики MAE: {mae_score}")

# Тестирование метрики MSE (Mean Squared Error)
model_mse = MyLineReg(n_iter=50, learning_rate=0.1, metric='mse')
model_mse.fit(X_df, y_series, verbose=False)
mse_score = model_mse.get_best_score()
print(f"Последнее значение метрики MSE: {mse_score}")

# Тестирование метрики RMSE (Root Mean Squared Error)
model_rmse = MyLineReg(n_iter=50, learning_rate=0.1, metric='rmse')
model_rmse.fit(X_df, y_series, verbose=False)
rmse_score = model_rmse.get_best_score()
print(f"Последнее значение метрики RMSE: {rmse_score}")

# Тестирование метрики MAPE (Mean Absolute Percentage Error)
model_mape = MyLineReg(n_iter=50, learning_rate=0.1, metric='mape')
model_mape.fit(X_df, y_series, verbose=False)
mape_score = model_mape.get_best_score()
print(f"Последнее значение метрики MAPE: {mape_score}")

# Тестирование метрики R2 (Coefficient of Determination)
model_r2 = MyLineReg(n_iter=50, learning_rate=0.1, metric='r2')
model_r2.fit(X_df, y_series, verbose=False)
r2_score = model_r2.get_best_score()
print(f"Последнее значение метрики R2: {r2_score}")

Последнее значение метрики MAE: 0.14274966283329524
Последнее значение метрики MSE: 0.033290942780611864
Последнее значение метрики RMSE: 0.18245805759300374
Последнее значение метрики MAPE: 0.45358307834809586
Последнее значение метрики R2: 0.9999970699329948


In [22]:
#Тестирование регуляризации
X, y = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X_df = pd.DataFrame(X)
y_series = pd.Series(y)

# Тестирование с регуляризацией L1
model_l1 = MyLineReg(n_iter=50, learning_rate=0.1, reg='l1', l1_coef=0.1, metric='mse')
model_l1.fit(X_df, y_series, verbose=10)
print("Последнее значение метрики MSE с L1 регуляризацией:", model_l1.get_best_score())

# Тестирование с регуляризацией L2
model_l2 = MyLineReg(n_iter=50, learning_rate=0.1, reg='l2', l2_coef=0.1, metric='mse')
model_l2.fit(X_df, y_series, verbose=10)
print("Последнее значение метрики MSE с L2 регуляризацией:", model_l2.get_best_score())

# # Тестирование с регуляризацией ElasticNet
model_elasticnet = MyLineReg(n_iter=50, learning_rate=0.1, reg='elasticnet', l1_coef=0.05, l2_coef=0.05, metric='mse')
model_elasticnet.fit(X_df, y_series, verbose=10)
print("Последнее значение метрики MSE с ElasticNet регуляризацией:", model_elasticnet.get_best_score())

# Тестирование модели без регуляризации (reg=None)
model_none = MyLineReg(n_iter=50, learning_rate=0.1, reg=None, metric='mse')
model_none.fit(X_df, y_series, verbose=10)
print("Последнее значение метрики MSE без регуляризации:", model_none.get_best_score())

start | loss: 11154.068597347617 | mse: 11153.868597347617
10 | loss: 16.328517555084016 | mse: 0.1975168197412967
20 | loss: 16.216514193918208 | mse: 0.06565448934844247
30 | loss: 16.20870056151608 | mse: 0.0555079792364534
40 | loss: 16.20742011587521 | mse: 0.05380972088026173
Последнее значение метрики MSE с L1 регуляризацией: 0.05349958692014
start | loss: 11154.068597347617 | mse: 11153.868597347617
10 | loss: 1178.4803108817116 | mse: 126.32420624639657
20 | loss: 1178.4679461509118 | mse: 125.3698940058319
30 | loss: 1178.4671988136552 | mse: 125.26505112366868
40 | loss: 1178.4670790943221 | mse: 125.24648422036164
Последнее значение метрики MSE с L2 регуляризацией: 125.24305757363275
start | loss: 11154.068597347617 | mse: 11153.868597347617
10 | loss: 629.8700499961899 | mse: 36.99205060759622
20 | loss: 629.831694376977 | mse: 36.019351640639044
30 | loss: 629.8292070977457 | mse: 35.9103600791985
40 | loss: 629.8288042127872 | mse: 35.89098295960591
Последнее значение ме